In [ ]:
#| default_exp helpers

In [ ]:
#| export
from __future__ import annotations

# Helpers

> ...

# Prologue

In [ ]:
#| export

import dataclasses
import functools
import importlib
import os
import sys
from binascii import hexlify
from functools import cache
from functools import partial
from pathlib import Path
from types import ModuleType
from typing import Any
from typing import DefaultDict

import fastcore.all as FC
from olio.common import Config


In [ ]:
import json
import operator
from functools import reduce
from inspect import Parameter

from fastcore.foundation import L
from fastcore.test import *
from olio.common import empty
from olio.common import setup_console
from olio.test import test_raises

----

In [ ]:
console, cprint = setup_console(140)

----

# Helpers

In [ ]:
#| export

emptyd, emptyl, emptyt = {}, [], ()

# BridgeCfg


In [ ]:
#| export

# @dataclasses.dataclass
class BridgeCfg(Config):
    """
    Settings for core `Bridget` behavior.
    
    if `True`:
    - `auto_show`: FastHTML objects display as HTML instead of markdown.
    - `auto_mount`: components with routes are automatically mounted.
    - `auto_id`: display elements get auto-generated IDs.
    - `bootstrap`: load bridget.js on import.
    - `current_did`: the ID of the current display cell.
    - `debug_req`: request debugging is enabled.
    """
    auto_show: bool = False
    auto_mount: bool = False
    auto_id: bool = False
    bootstrap: bool = os.environ.get('BRIDGET_BOOTSTRAP', '').lower() in ('true', '1', 'on', 'yes', 'y')
    current_did: str|None = None
    debug_req: bool = False

bridge_cfg = BridgeCfg()

In [ ]:
bridge_cfg.show()

{}


# _get_globals


In [ ]:
#| exporti

def _get_globals(mod: str):
    if hasattr(sys, '_getframe'):
        glb = sys._getframe(2).f_globals
    else:
        glb = sys.modules[mod].__dict__
    return glb

In [ ]:
def _gtest(): return _get_globals(__name__)
g1 = _gtest()
g2 = globals()
test_eq(g1, g2)

# Bundle path

In [ ]:
#| export

def bundle_path(mod:str|ModuleType):
    "Return the path to the module's directory or current directory."
    if isinstance(mod, str): mod = importlib.import_module(mod)
    return Path(fn).parent if (fn := getattr(mod, '__file__', None)) else Path()

In [ ]:
import bridget

test_eq(bundle_path(__name__), Path('.'))
test_eq(bundle_path('bridget').resolve(), Path(bridget.__file__).parent)

# run_command

In [ ]:
#| export

async def arun_command(command: str, cwd: Path|None=None, **kwargs):
    import anyio
    import subprocess
    try:
        process = await anyio.run_process(
            command,
            cwd=cwd or Path().absolute().parent,
            **kwargs
        )
        return process.stdout.decode('utf-8'), process.stderr.decode('utf-8')
    except subprocess.CalledProcessError as e:
        return e.stdout.decode('utf-8'), e.stderr.decode('utf-8')

def run_command(command: str, cwd: Path|None=None, **kwargs):
    import subprocess
    result = subprocess.run(
        command,
        shell=True,
        capture_output=True,
        text=True,
        cwd=cwd or Path().absolute().parent,
        check=True,
        **kwargs
    )
    if result.returncode != 0:
        raise RuntimeError(result.stderr)
    return result.stdout, result.stderr

In [ ]:
a,_ = await arun_command('ls')
test_eq('bridget' in a, True)
_,b = await arun_command('node notfound')
test_eq('Error: Cannot find module' in b, True)

with test_raises(FileNotFoundError):
    await arun_command('ls', cwd=Path('/not/found'))

# Singleling
> Basic, we're-all-adults-here, singleton.

Should be used as a mixin, first base in subclasses..

In [ ]:
#| export

def _noop(*args, **kwargs): pass
class Singleling:
    def __new__(cls, *args, **kwargs):
        if '__instance__' not in cls.__dict__: cls.__instance__ = super().__new__(cls, *args, **kwargs)
        cls.__instance__.__init__(*args, **kwargs)
        setattr(type(cls.__instance__), '__init__', _noop)
        return cls.__instance__

In [ ]:
class TestSingle(Singleling):
    def __init__(self):
        self.a = 1

test_is(o := TestSingle(), TestSingle())
test_eq(o.a, 1)

# Kounter
> Counter of keys

In [ ]:
#| export

class Kounter:
    def __init__(self): self.d = DefaultDict(int)
    def __call__(self, k): d = self.d; d[k] += 1; return self.d[k]

kounter = Kounter()

In [ ]:
cntr = Kounter()
cntr('a')
cntr('b')
cntr('a')
cntr('a')
cntr('b')
cntr('b')
cntr('b')
test_eq(cntr.d, {'a': 3, 'b': 4})
test_eq(cntr('int'), 1)

# id_gen
> Generate unique IDs for HTML elements


In [ ]:
import random
import re
from pathlib import Path

lines = Path("static/wordlist.txt").read_text().splitlines()
words = [line.strip() for line in lines if line.isalpha()]

In [ ]:
def modify_word(word):
    # Randomly capitalize the first or second letter
    if len(word) > 1:
        idx_to_capitalize = random.choice([0, 1])
        word = word[:idx_to_capitalize] + word[idx_to_capitalize].upper() + word[idx_to_capitalize + 1:]
    else:
        word = word.upper()  # If single letter, capitalize it
    
    # Randomly add a number (0–99) at the start or end
    if random.choice([True, False]):
        number = random.randint(0, 99)
        # if random.choice([True, False]):
        #     word = f"{number}{word}"  # Number at the start
        # else:
        word = f"{word}{number}"  # Number at the end
    
    return word

def generate_readable_id(num_words=3):
    words_part = [modify_word(random.choice(words)) for _ in range(num_words)]
    id_candidate = '-'.join(words_part)

    # Ensure it's a valid CSS identifier
    if not re.match(r"^[a-zA-Z_][\w\-]*$", id_candidate):  # Add '_' if invalid
        id_candidate = f"_{id_candidate}"
    
    return f"{id_candidate}-{random.randint(0, 9999)}"

In [ ]:
generate_readable_id(), generate_readable_id()

('Neither58-Seeing-bRand71-8172', 'Dealt84-Cums-hIding-5379')

In [ ]:
#| export

def simple_id():
    return 'b'+hexlify(os.urandom(16), '-', 4).decode('ascii')

def id_gen():
    kntr = Kounter()
    def _(o:Any=None): 
        if o is None: return simple_id()
        # return f"{type(o).__name__}_{hash(o) if isinstance(o, Hashable) else kntr(type(o).__name__)}"
        return f"{type(o).__name__}_{kntr(type(o).__name__)}"
    return _

The `id_gen` function creates a function that takes any object and generates an unique Id valid during the current session. Useful for creating unique element IDs in dynamic HTML content.


In [ ]:
new_id = id_gen()
new_id(), new_id()

('b6c494805-5125c235-b36a46c6-f53c2665',
 'b4b9e3e20-55445da2-8787d197-4a20f41c')

In [ ]:
int_id = id_gen()
int_id(7), int_id(8)

('int_1', 'int_2')

In [ ]:
obj_id = id_gen()
o1, o2 = object(), object()
print(obj_id(o1), obj_id(o2))

dict_id = id_gen()
print(dict_id(d1 := {'a': 1}), dict_id(d2 := {'a': 1}))

pth_id = id_gen()
print(pth_id(Path('.')), pth_id(Path()), pth_id(Path('./bin')))

object_1 object_2
dict_1 dict_2
PosixPath_1 PosixPath_2 PosixPath_3


# patch_cached

In [ ]:
#| export

def patch_cached(cls, f, name:str|None=None):
    name = name or (f if not isinstance(f, partial) else f.func).__name__ 
    setattr(cls, name, cache(f))

In [ ]:
# type: ignore

from collections import defaultdict

class Test:
    def __init__(self): self.d = defaultdict(list)

def a(self, n:int=1):
    self.d['a'].append(n)
    return n+1


patch_cached(Test, a)

t1 = Test()
test_eq(t1.a(), 2)
test_eq(t1.a(), t1.a())
test_eq(t1.d['a'], [1])
test_eq(t1.a(3), 4)
test_eq(t1.a(3), t1.a(3))
test_eq(t1.d['a'], [1, 3])

t2 = Test()
test_eq(t2.a(), 2)
test_eq(t2.a(), t2.a())
test_eq(t2.d['a'], [1])
test_eq(t2.a(7), 8)
test_eq(t2.a(7), t2.a(7))
test_eq(t2.d['a'], [1, 7])

# patch_cached_property

In [ ]:
#| export

def patch_cached_property(cls, f, name:str|None=None):
    is_partial, prop = isinstance(f, partial), functools.cached_property(f)
    if is_partial: prop.__doc__ = f.func.__doc__
    prop.attrname = name or (f if not is_partial else f.func).__name__ 
    setattr(cls, prop.attrname, prop)

In [ ]:
# type: ignore

def a(self): 
    "a docs"
    self.d['a'].append('a'); return 2
def _b(self, n): 
    "b docs"
    self.d['b'].append(n); return n+n
class Test: 
    def __init__(self): self.d = defaultdict(list)

patch_cached_property(Test, a)
patch_cached_property(Test, lambda self: _b(self, 2), 'b2')
patch_cached_property(Test, partial(_b, n=3), 'b3')
patch_cached_property(Test, partial(lambda self: _b(self, 4)), 'b4')

t1 = Test()
test_eq(t1.a, 2)
test_eq(t1.a, t1.a)
test_eq(t1.d['a'], ['a'])
test_eq(t1.b2, 4)
test_eq(t1.b2, t1.b2)
test_eq(t1.d['b'], [2])
test_eq(t1.b3, 6)
test_eq(t1.b3, t1.b3)
test_eq(t1.d['b'], [2, 3])
test_eq(t1.b4, 8)
test_eq(t1.b4, t1.b4)
test_eq(t1.d['b'], [2, 3, 4])

In [ ]:
#| export

class cached_property(functools.cached_property):
    def __init__(self, func):
        super().__init__(func)
        for o in functools.WRAPPER_ASSIGNMENTS: setattr(self, o, getattr(func, o))
    # def __set_name__(self, owner, name):
    #     super().__set_name__(owner, name)
    #     if self.attrname is None:
    #         self.__qualname__ = f"{owner.__name__}.{name}"

# bridge_metadata

In [ ]:
#| export

def bridge_metadata(metadata:dict|None=None, **kwargs):
    if not metadata: metadata = {'bridge': {**kwargs}}
    elif not 'bridge' in metadata: metadata['bridge'] = {**kwargs}
    else: metadata['bridge'].update(**kwargs)
    return metadata

def skip(metadata:dict|None=None, **kwargs): return bridge_metadata(metadata, skip=True, **kwargs)

In [ ]:
test_eq(bridge_metadata(), {'bridge': {}})
test_eq(bridge_metadata(skip=True), {'bridge': {'skip': True}})
test_eq(bridge_metadata({'autoshow': True}, skip=True), {'autoshow': True, 'bridge': {'skip': True}})
test_eq(skip(), {'bridge': {'skip': True}})
test_eq(skip({'bridge': {'auto_show': True}}), {'bridge': {'skip': True, 'auto_show': True}})
test_eq(skip(auto_show=True), {'bridge': {'skip': True, 'auto_show': True}})


# compose_first
> like [fastcore.compose](https://fastcore.fast.ai/basics.html#compose), but args are passed only to first function

In [ ]:
#| export

def compose_first(*funcs, order=None):
    "Create a function that composes all functions in `funcs`, passing along remaining `*args` and `**kwargs` to first function"
    funcs = FC.listify(funcs)
    if len(funcs)==0: return FC.noop
    if len(funcs)==1: return funcs[0]
    if order is not None: funcs = FC.sorted_ex(funcs, key=order)
    def _inner(x, *args, **kwargs):
        x = funcs[0](x, *args, **kwargs)  # type: ignore
        for f in funcs[1:]: x = f(x)  # type: ignore
        return x
    return _inner


# Colophon
----

In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '01_helpers.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)